In [1]:
from pyspark.sql import SparkSession
# from datetime import datetime, date
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
import os
from numpy import nan
from pyspark.sql import Row
from pyspark.sql.functions import lit,regexp_replace, col, trim, regexp_extract
spark = SparkSession.builder.appName("Masters").getOrCreate()
# spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
# pd.set_option('display.max_rows', 30)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', 10)
# pd.set_option('display.max_colwidth', None)




your 131072x1 screen size is bogus. expect trouble


23/03/03 08:19:32 WARN Utils: Your hostname, ZenBook-win resolves to a loopback address: 127.0.1.1; using 172.18.249.83 instead (on interface eth0)
23/03/03 08:19:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/03 08:19:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
taxonomy_excel = pd.read_excel("/home/rszczygielski/bioinf/magisterka/geneBank/taxonomy1.xlsx")
array_of_values = taxonomy_excel.values
list_of_values = []
for row_array in array_of_values:
    row_array = row_array[~pd.isnull(row_array)]
    row_list = list(row_array)
    list_of_values.append(row_list)


taxonomy_row_list = []
for elem in list_of_values:
    new_elem = list(map(str.strip, elem))
    taxonomy_row_list.append(new_elem)
    # print(new_elem)
taxonomy_row_list[0]


["'Chordata'",
 "'Craniata'",
 "'Vertebrata'",
 "'Chondrichthyes'",
 "'Elasmobranchii'"]

In [3]:
df_masters = pd.read_csv("/home/rszczygielski/bioinf/magisterka/geneBank/results/Organisms_mitochondion_2.txt",\
    header=0,\
    delimiter='\t',\
    usecols=["ACCESSION", "ORGANISM","TAXONOMY", "PREVIOUS_GENE","CONTROL_REGION", "NEXT_GENE"],\
    index_col="ACCESSION")




In [29]:
def get_gene_info(aggragated_taxonomy, gene_column_name):
    gen_info_list = aggragated_taxonomy[gene_column_name].unique().tolist()
    if len(gen_info_list) == 0:
        return "Not found"
    splited_gen_info = np.char.split(gen_info_list, " ")
    gene_set = set()
    specific_gene_set = set()
    if gene_column_name == "PREVIOUS_GENE":
        gene_index = 8
    else:
        gene_index = 0
    for gene in splited_gen_info:
        gene_set.add(gene[gene_index])
        specific_gene_set.add(gene[-1])
    joind_gene = ", ".join(list(gene_set))
    joind_specific_gene_set = ", ".join(list(specific_gene_set))
    return [joind_gene, joind_specific_gene_set]

df_taxonomy = df_masters[["TAXONOMY", "PREVIOUS_GENE","CONTROL_REGION", "NEXT_GENE"]]

for row_excel_taxonomy in enumerate(taxonomy_row_list):
    joined_values_entry = ", ".join(row_excel_taxonomy[1])
    aggragated_taxonomy = df_taxonomy.where(df_taxonomy["TAXONOMY"].str.contains(joined_values_entry)).dropna()
    # aggragated_taxonomy.to_excel(f"/home/rszczygielski/bioinf/magisterka/geneBank/results/aggregations/{joined_values_entry}.xlsx")
    previos_gene = get_gene_info(aggragated_taxonomy, "PREVIOUS_GENE")
    control_region = get_gene_info(aggragated_taxonomy, "CONTROL_REGION")
    next_gene = get_gene_info(aggragated_taxonomy, "NEXT_GENE")
    gene_info_df = pd.DataFrame(data=[[joined_values_entry, previos_gene[0], previos_gene[1], control_region[0], control_region[1], next_gene[0], next_gene[1]]],
                                columns=["TAXNONOMY", "PREVIOUS_GENE_PRODUCT","PREVIOUS_GENE_SPECIFIC_PRODUCT",
                                "CONTROL_REGION_PRODUCT", "CONTROL_REGION_SPECIFIC_PRODUCT", "NEXT_GENE_PRODUCT", "NEXT_GENE_SPECIFIC_PRODUCT"])
    gene_info_df.to_excel(f"/home/rszczygielski/bioinf/magisterka/geneBank/results/aggregations/gene_info/GENE_INFO_{joined_values_entry}.xlsx")


In [ ]:
test = ", ".join(taxonomy_row_list[6])
aggragated_taxonomy = df_taxonomy.where(df_taxonomy["TAXONOMY"].str.contains(test)).dropna()
len(aggragated_taxonomy)
gen_info_list = aggragated_taxonomy["PREVIOUS_GENE"].unique().tolist()
splited_gen_info = np.char.split(gen_info_list, " ")
gene_set = set()
specific_gene_set = set()
for gene in splited_gen_info:
    print(gene)
    gene_set.add(gene[8])
    specific_gene_set.add(gene[-1])





def get_gene_info(aggragated_taxonomy, gene_column_name):
    gen_info_list = aggragated_taxonomy[gene_column_name].unique().tolist()
    splited_gen_info = np.char.split(gen_info_list, " ")
    gene_set = set()
    specific_gene_set = set()
    for gene in splited_gen_info:
        gene_set.add(gene[8])
        specific_gene_set.add(gene[-1])
    return gene_set, specific_gene_set


9

['', '', '', '', '', '', '', '', 'tRNA', '[15535:15605](-)', "['tRNA-Pro']"]
['', '', '', '', '', '', '', '', 'tRNA', '[15485:15554](-)', "['tRNA-Pro']"]
['', '', '', '', '', '', '', '', 'tRNA', '[15488:15556](-)', "['tRNA-Pro']"]
['', '', '', '', '', '', '', '', 'tRNA', '[15492:15562](-)', "['tRNA-Pro']"]
['', '', '', '', '', '', '', '', 'tRNA', '[15486:15555](-)', "['tRNA-Pro']"]
['', '', '', '', '', '', '', '', 'tRNA', '[15417:15486](-)', "['tRNA-Pro']"]
['', '', '', '', '', '', '', '', 'tRNA', '[15494:15564](-)', "['tRNA-Pro']"]
['', '', '', '', '', '', '', '', 'tRNA', '[15481:15550](-)', "['tRNA-Pro']"]
['', '', '', '', '', '', '', '', 'tRNA', '[15482:15551](-)', "['tRNA-Pro']"]


In [ ]:

df_masters = spark.read.option('delimiter', '\t').csv("/home/rszczygielski/bioinf/magisterka/geneBank/results/Organisms_mitochondion_1.txt", header=True)
